# 元编程

这部分还重点参考了这个资料：[Python Metaclasses](https://realpython.com/python-metaclasses/)

术语元编程是指程序具有了解或操纵自身的潜力。Python支持一种称为metaclasses的类的元编程形式。

元类是一个比较复杂的面向对象概念，几乎隐藏在所有Python代码之后。无论是否知道，你都在使用它们。在大多数情况下，无需意识到这一点。大多数Python使用者很少（即使有的话）也不必考虑元类。

但是，当需要时，Python提供了并非所有面向对象的语言都支持的功能：我们可以深入了解并自定义元类。自定义元类的使用引起了一些争议，正如Python 禅意作者Tim Peters所引用的那样：

> 元类具有比99％的用户应该担心的更深的魔力。如果您想知道是否需要它们，则不需要（实际上需要它们的人肯定会知道他们需要它们，并且不需要解释原因）。”
— 蒂姆·彼得斯

有一些python使用者则认为永远不要使用自定义元类。这可能有点夸张，但是确实很可能不需要自定义元类。如果不是很明显有问题需要解决，如果能以更简单的方式解决问题，那么不用它可能会让代码更简洁，更易读。

尽管如此，理解 Python 元类还是值得的，因为通过元类可以更好地理解Python 类的内部。可能有一天会遇到一种情况：只需要一个自定义元类即可以解决问题。

## 两种类 NewClass Vs ClassicClass

Python类可以是以下两个变体之一：ClassicClass 或 NewClass，由于尚未确定官方术语，因此将它们非正式地称为旧类和新类。

对于旧类，类和类型不是一回事。旧类的实例始终由称为instance的一个内置类型实现。如果obj是旧类的一个实例，那么obl.\_\_class\_\_ 则给出了类，但是 type(obj) 则是instance。比如在python 2.7里面：

```Python
>>> class Foo:
...     pass
...
>>> x = Foo()
>>> x.__class__
<class __main__.Foo at 0x000000000535CC48>
>>> type(x)
<type 'instance'>
```

新类则统一了类和类型的概念。如果obj是新类的实例，那么type(obj)和obj.\_\_class\_\_ 是一样的

In [1]:
class Foo:
    pass
obj = Foo()
obj.__class__

__main__.Foo

In [2]:
type(obj)

__main__.Foo

In [3]:
n = 5
d = { 'x' : 1, 'y' : 2 }

class Foo:
    pass

x = Foo()

for obj in (n, d, x):
    print(type(obj) is obj.__class__)

True
True
True


## 类和类型

Python3中所有类都是新类。因此在Python 3中，对象的类型和类是可以互换着使用的。Python2.2之前是不支持新类的，之后python2默认的也是旧类。

记住，在python中，所有都是对象，类也是对象。所以类也得有类型。

In [4]:
class Foo:
    pass

x = Foo()

type(x)

__main__.Foo

In [5]:
type(Foo)

type

x的类型是class Foo。但是Foo类本身的类型是type。通常，新式类的类型都是type。

那些都比较熟悉的内置类的类型也是type，包括type自己：

In [6]:
for t in int, float, dict, list, tuple:
    print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [7]:
type(type)

type

type就是一个元类，其中的类是实例。就像普通对象是类的实例一样，Python中的任何新类或者说Python 3中的任何类都是type这一元类的实例。

![](pictures/class-chain.5cb031a299fe.png)

## 动态定义一个类

当传递一个参数时，内置函数type()将返回对象的类型。对于新类，通常与对象的\_\_class\_\_属性相同：

In [8]:
type(3)

int

In [9]:
type(['foo', 'bar', 'baz'])

list

In [10]:
t = (1, 2, 3, 4, 5)
type(t)

tuple

In [11]:
class Foo:
    pass

type(Foo())

__main__.Foo

还可以使用三参数调用的方式：type(<name>, <bases>, <dct>)：

- <name>指定类名称。该类的__name__属性。
- <bases>指定类继承的基类的元组。该类的__bases__属性。
- <dct>指定一个包含类主体定义的命名空间字典。该类的__dict__属性。

以这种方式调用type()会创建type元类的新实例。换句话说，它动态创建一个新类。

下面给出一些示例。每个示例最上面的代码段使用type()来动态定义一个类，而下面的代码段则使用class语句以通常的方式定义该类。每个例子中，这两个代码段在功能上是等效的。

在第一个示例中，传递给type()的<bases>和<dct>参数均为空。没有指定任何父类的继承，并且最初在命名空间字典中未放置任何内容。这是最简单的类定义：

In [12]:
Foo = type('Foo', (), {})

x = Foo()
x

In [13]:
class Foo:
    pass

x = Foo()
x

例2里<bases>是一个具有单个元素Foo的元组，指定Bar继承的父类。属性attr放置在名称空间字典中：

In [14]:
Bar = type('Bar', (Foo,), dict(attr=100))

x = Bar()
print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


In [15]:
class Bar(Foo):
    attr = 100


x = Bar()
print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


这次，<bases>又是空的。通过<dct>参数将两个对象放入命名空间字典中。第一个是名为attr的属性，第二个是名为attr_val的函数，该函数成为已定义类的方法：

In [16]:
Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x : x.attr
    }
)

x = Foo()
print(x.attr)
print(x.attr_val())

100
100


In [17]:
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr


x = Foo()
print(x.attr)
print(x.attr_val())

100
100


lambda在Python中只能定义非常简单的函数。在下面的示例中，在外部定义了一个稍微复杂一点的函数，然后通过名称attr_val在名称空间字典中将f分配给它：

In [18]:
def f(obj):
    print('attr =', obj.attr)

Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': f
    }
)

x = Foo()
print(x.attr)
x.attr_val()

100
attr = 100


In [19]:
def f(obj):
    print('attr =', obj.attr)

class Foo:
    attr = 100
    attr_val = f


x = Foo()
print(x.attr)
x.attr_val()

100
attr = 100


## 定制元类

再次考虑简单的Foo：

In [20]:
class Foo:
    pass

f = Foo()

表达式Foo()创建了类Foo的新实例。解释器遇到Foo()时，将发生以下情况：

- Foo的父类的\_\_call\_\_()方法被调用。由于Foo是标准的新类，因此其父类是type元类，因此调用type的\_\_call\_\_()方法。
- 该\_\_call\_\_()方法依次调用以下内容：
    - \_\_new\_\_()
    - \_\_init\_\_()
    
如果Foo未定义\_\_new\_\_()和\_\_init\_\_()，则默认方法继承自Foo的祖先。如果Foo中确实定义了这些方法，则它们会覆盖祖先中的方法，从而在实例化Foo时允许自定义行为。

在下面，定义了一个自定义方法new()，并将其指定为Foo的\_\_new\_\_()方法：

In [21]:
def new(cls):
    x = object.__new__(cls)
    x.attr = 100
    return x

Foo.__new__ = new

f = Foo()
f.attr

100

In [22]:
g = Foo()
g.attr

100

这会修改类Foo的实例化行为：每次Foo创建实例时，默认情况下都会使用名为attr的属性对其进行初始化，该属性的值为100。（这样的代码通常会出现在\_\_init\_\_()方法中，而通常不会出现在方法\_\_new\_\_()中，这个示例是为演示目的而设计的）

现在，正如已经重申的，类也是对象。假设您要在创建类似的类时，可以以类似的自定义方式完成Foo实例化行为。如果要遵循上述模式，需要再次定义一个自定义方法，并将其分配为\_\_new\_\_()方法。Foo是type元类的实例，因此代码如下所示：

In [23]:
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

如您所见，报错了，不能重新赋给元类type的\_\_new\_\_()方法。Python不允许这样做。

因为type是派生所有新类的元类。无论如何，不能这么做。但是，如果要自定义类的实例化，有什么办法？

一种可能的解决方案是自定义元类。该元类是从type派生的，然后就可以使用这个元类了。

第一步是定义一个从type派生的元类，如下所示：

In [24]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

“class Meta(type):”语句指定Meta从type派生。由于type是一个元类，因此现在构建了一个Meta元类。

请注意，已为Meta定义了自定义方法\_\_new\_\_()。无法直接对元类type执行此操作。该\_\_new\_\_()方法执行以下操作：

- 用父元类（即type）的代理--super()的__new__()方法创建一个新的类
- 将自定义属性attr分配给该类，其值为100
- 返回新创建的类

下面定义一个新类Foo，并指定其元类是自定义元类Meta，而不是标准元类type。使用类定义中的关键字metaclass来完成此操作，如下所示：

In [25]:
class Foo(metaclass=Meta):
    pass

Foo.attr

100

现在， Foo已经自动拿到Meta元类的attr属性。当然，类似定义的任何其他类也将这样做：

In [26]:
class Bar(metaclass=Meta):
    pass

class Qux(metaclass=Meta):
    pass

Bar.attr, Qux.attr

(100, 100)

与类充当创建对象的模板的方式相似，元类充当创建类的模板。元类有时称为类工厂。

比较以下两个示例：

对象工厂：

In [27]:
class Foo:
    def __init__(self):
        self.attr = 100


x = Foo()
print(x.attr)


y = Foo()
print(y.attr)


z = Foo()
print(z.attr)

100
100
100


类工厂：

In [28]:
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100

class X(metaclass=Meta):
    pass

print(X.attr)


class Y(metaclass=Meta):
    pass

print(Y.attr)


class Z(metaclass=Meta):
    pass

print(Z.attr)

100
100
100


## 真的需要元类么

上面的类工厂示例是元类如何工作的本质。它们允许自定义的类实例化。这个好处就是不必重复地去为每个类写attr，代码能更好地复用。

但是尽管如此，在每个新创建的类上赋予自定义attr属性仍然有很多麻烦。我们真的需要一个元类吗？

在 Python 中，至少有几种其他方法可以有效地完成同一件事：

In [29]:
class Base:
    attr = 100


class X(Base):
    pass


class Y(Base):
    pass


class Z(Base):
    pass


print(X.attr)

print(Y.attr)

print(Z.attr)


100
100
100


In [30]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class X:
    pass

@decorator
class Y:
    pass

@decorator
class Z:
    pass


print(X.attr)

print(Y.attr)

print(Z.attr)

100
100
100


正如蒂姆·彼得斯（Tim Peters）所建议的那样，元类很容易进入“从问题中寻找解决方案”的境界。通常不需要创建自定义元类。如果眼前的问题可以用更简单的方法解决，那就应该这样解决。尽管如此，理解元类还是有好处的，这样您就可以大致理解Python类，并可以识别何时才真正适合使用元类。

装饰器是较常用的元编程方式，下面再看点例子。

## 装饰器

比如想在函数上添加一个装饰器，增加额外的操作处理(比如日志、计时等)。这是很常用的功能。使用额外的代码包装一个函数，可以定义一个装饰器函数：

In [31]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [32]:
@timethis
def countdown(n):
#       Counts down
    while n > 0:
        n -= 1

In [33]:
countdown(100000)

countdown 0.006994724273681641


实际上一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数。下面两个函数是等价的：

In [34]:
@timethis
def countdown(n):
    pass

In [35]:
def countdown(n):
    pass
countdown = timethis(countdown)

另外，内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。

不过在使用装饰器时，要注意复制元信息。即任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数。如果你忘记了使用 @wraps ， 那么你会发现被装饰函数丢失了所有有用的信息。

还可以定义带参数的装饰器，比如你想写一个装饰器，给函数添加日志功能，同时允许用户指定日志的级别和其他的选项。 下面是这个装饰器的定义和使用示例：

In [36]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')